In [2]:
import http.server
import socketserver
import json
import io  # For handling audio data
import os 
# Import libraries for database and speech recognition (install if needed)
import mysql.connector  # pip install mysql-connector-python
import speech_recognition as sr  # pip install SpeechRecognition
from flask import Flask, request, jsonify

app = Flask(__name__)
# Database connection details (replace with your actual values)
DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "Anu@123"
DB_NAME = "dhwani"


PORT = 8000  # Change port if needed

class Handler(http.server.BaseHTTPRequestHandler):

    # Function to establish a connection to the MySQL database
    def connect_to_db(self):
        try:
            mydb = mysql.connector.connect(
                host=DB_HOST,
                user=DB_USER,
                password=DB_PASSWORD,
                database=DB_NAME
            )
            return mydb
        except mysql.connector.Error as err:
            print("Error connecting to database:", err)
            return None

    # Function to close the database connection
    def close_connection(self, mydb):
        if mydb:
            mydb.close()

    # Function to validate user login credentials (replace with your logic)
    def validate_login(self, data):
        username = data.get("username")
        password = data.get("password")

        # Implement logic to check username and password against database
        # using SQL queries with your connected database object (from connect_to_db())
        # This is a placeholder, replace with your actual validation logic
        return username == "user" and password == "password"

    # Function to process speech data
    def process_speech(self, audio_data):
        recognizer = sr.Recognizer()
        with sr.AudioFile(io.BytesIO(audio_data)) as source:
            audio = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio)
            return text
        except sr.UnknownValueError:
            return "Could not understand audio"
        except sr.RequestError as e:
            return f"Recognition service error: {e}"

    def do_GET(self):
        if self.path == '/':
            self.path = '/index.html'  # Serve index.html by default

        try:
            # Open the requested file in binary mode
            with open(os.path.join(os.getcwd(), self.path[1:]), 'rb') as file:
                # Determine the content type based on the file extension
                if self.path.endswith('.html'):
                    content_type = 'text/html'
                elif self.path.endswith('.css'):
                    content_type = 'text/css'
                else:
                    content_type = 'application/octet-stream'  # Default to binary data if content type is unknown

                # Send the response with appropriate content type
                self.send_response(200)
                self.send_header('Content-type', content_type)
                self.end_headers()

                # Write the content of the file to the response
                self.wfile.write(file.read())

        except IOError:
            # If the file is not found, send a 404 error
            self.send_error(404, 'File Not Found')


    def do_POST(self):
        self.send_response(200)
        content_length = int(self.headers['Content-Length'])  # Get data length
        post_data = self.rfile.read(content_length).decode('utf-8')

        # Check for login or speech data based on the path
        if self.path == '/login':
            mydb = self.connect_to_db()
            if mydb:
                login_success = self.validate_login(json.loads(post_data))
                self.close_connection(mydb)
                response = jsonify({'success': login_success})
            else:
                response = jsonify({'error': 'Database connection failed'})
        elif self.path == '/speak':
            text = self.process_speech(post_data)
            response = jsonify({'text': text})
        else:
            response = jsonify({'error': 'Invalid endpoint'})

        self.wfile.write(response.encode())  # Send JSON response
        @app.route('/chat', methods=['POST'])
        def chat():
            data = request.json
            message = data.get('message')

            def process_message(message):
            # Convert the message to lowercase for easier processing
                message = message.lower()

                # Define some basic responses for certain keywords or phrases
                if 'hello' in message or 'hi' in message:
                    return 'Hi there! How can I assist you today?'
                elif 'how are you' in message:
                    return "I'm just a computer program, but thanks for asking!"
                elif 'bye' in message:
                    return 'Goodbye! Have a great day!'
                else:
                    # If no specific keyword is matched, provide a default response
                    return "I'm sorry, I didn't understand that. Can you please rephrase?"
                
        @app.route('/process_message', methods=['POST'])
       
        def handle_message():
            if request.method == 'POST':
                data = request.get_json()
                user_message = data.get('message')
                if user_message:
                    # Process the user message
                    response = process_message(user_message)
                    return jsonify({'response': response})
                else:
                    return jsonify({'error': 'No message provided'})
            else:
                return jsonify({'error': 'Method not allowed'})

        if __name__ == '__main__':
            app.run(port=8000)



with socketserver.TCPServer(("", PORT), Handler) as httpd:
    print("Serving at port", PORT)
    httpd.serve_forever()


Serving at port 8000


127.0.0.1 - - [18/Mar/2024 15:52:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 15:52:11] "GET /style.css HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 15:52:11] "GET /bg.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 15:52:15] "GET /login.html HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 15:52:15] "GET /style.css HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 15:52:15] "GET /bg.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 15:52:32] "GET /chatbot.html HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 15:52:33] "GET /style.css HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 15:52:33] "GET /bg.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 15:52:37] "POST /speak HTTP/1.1" 200 -
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 58270)
Traceback (most recent call last):
  File "c:\Users\GPK\anaconda3\Lib\socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "c:\Users\GPK\anaconda3\Lib\so